# **Dependency Injection**
----

In [18]:
from pprint import pprint

from dicfg.factory import build_config
from dicfg.reader import ConfigReader


config_reader = ConfigReader(name='myconfig')

### To use dependency injection in our config we first need to setup a **project with object dependencies**

In [19]:
class ProjectComponent:
    def __init__(self, name):
        self.name = name


class MyProject:
    def __init__(self, project_component: ProjectComponent):
        self.project_component = project_component

### ObjectFactory can build objects with the **\*object** key

In [20]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                # a classes/functions can be build via the *object key
                "*object": "__main__.ProjectComponent",
                
                # any arguments can specified below
                "name": "my_project_component",
        
                # non-keyword arguments can be specified with *args, e.g.,
                # "*args": ["my_project_component"],
            },
        },
    },
}

In [21]:
config = config_reader.read(user_config)
pprint(config, sort_dicts=False)
objects = build_config(config["default"])

print("\n build objects: \n")

pprint(objects)

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'*object': '__main__.ProjectComponent',
                                   'name': 'my_project_component'}}}

 build objects: 

{'config_dict': {'sub_config': 'water'},
 'config_int': 1,
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_string': 'fire',
 'project_component': <__main__.ProjectComponent object at 0x7f4676edeb50>}


### ObjectFactory can **return types with the \*return_type:** key

In [22]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "*object": "__main__.ProjectComponent",
                # The value for *return_type can be anything.
                # However, it is highly recommended to use True as convention
                "*return_type": "True",
            },
        },
    },
}

In [23]:
config = config_reader.read(user_config)
pprint(config, sort_dicts=False)
objects = build_config(config["default"])

print("\n build objects: \n")

pprint(objects)

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'*object': '__main__.ProjectComponent',
                                   '*return_type': 'True'}}}

 build objects: 

{'config_dict': {'sub_config': 'water'},
 'config_int': 1,
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_string': 'fire',
 'project_component': <class '__main__.ProjectComponent'>}


### Dependency Injection can be done by **object interpolation**

In [24]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "*object": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "*object": "__main__.MyProject",
                
                # object interpolation
                "project_component": "${project_component}",
            },
        },
    },
}

In [25]:
config = config_reader.read(user_config)
pprint(config, sort_dicts=False)
objects = build_config(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)
print("\nproject component name:", objects["project"].project_component.name)

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'*object': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'*object': '__main__.MyProject',
                         'project_component': '${project_component}'}}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': <__main__.ProjectComponent object at 0x7f4676edb1c0>,
 'project': <__main__.MyProject object at 0x7f4676edbd00>}

project component name: my_project_component


### ObjectFactory interpolation of objects is done by **object reference**

In [26]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "object*": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "*object": "__main__.MyProject",
                
                # object interpolation by object reference
                "project_component": "${project_component}",
            },
            
            # object interpolation by object reference
            "my_interpolation_component": "${project_component}",
        },
    },
}

In [27]:
config = config_reader.read(user_config)
pprint(config, sort_dicts=False)
objects = build_config(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)

print("\nreference       project_component:", objects["project_component"])
print("reference interpolation_component:", objects["my_interpolation_component"])

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'object*': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'*object': '__main__.MyProject',
                         'project_component': '${project_component}'},
             'my_interpolation_component': '${project_component}'}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': {'object*': '__main__.ProjectComponent',
                       'name': 'my_project_component'},
 'project': <__main__.MyProject object at 0x7f4676ec0820>,
 'my_interpolation_component': {'object*': '__main__.ProjectComponent',
                                'name': 'my_project_component

### ObjectFactory allows for **object attribute interpolation**

In [28]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "*object": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "object*": "__main__.MyProject",
                "project_component": "${project_component}",
            },
            "my_attribute_component": "${project_component}",
            
            # object attribute interpolation
            "my_attribute_component_name": "${project_component.name}",  
        },
    },
}

In [29]:
config = config_reader.read(user_config)
pprint(config, sort_dicts=False)
objects = build_config(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)

print("\nmy_attribute_component_name:", objects["my_attribute_component_name"])

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'*object': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'object*': '__main__.MyProject',
                         'project_component': '${project_component}'},
             'my_attribute_component': '${project_component}',
             'my_attribute_component_name': '${project_component.name}'}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': <__main__.ProjectComponent object at 0x7f4676b87700>,
 'project': {'object*': '__main__.MyProject',
             'project_component': <__main__.ProjectComponent object at 0x7f4676b87700>},
 'my_attribute_component': <__m